In [1]:
"""
Для цієї лабораторної роботи я буду використовувати датасет CNN/DailyMail. Поля: text, highlights
Посилання: https://www.kaggle.com/datasets/gowrishankarp/newspaper-text-summarization-cnn-dailymail

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Я ВЗЯВ 10 тис. даних з датасету через те, що у мене зависає середовище під час навчання та навіть
перезавантажується комп'ютер + ЗМЕНШИВ КІЛЬКІСТЬ ТЕКСТІВ ДЛЯ СУМАРИЗАЦІЇ ДЛЯ ПАСИВНОГО ТА АКТИВНОГО НАВЧАННЯ.
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

Запуск веб-додатку ідентичний з попередніх лаб.
"""

"\nДля цієї лабораторної роботи я буду використовувати датасет CNN/DailyMail. Поля: text, highlights\nПосилання: https://www.kaggle.com/datasets/gowrishankarp/newspaper-text-summarization-cnn-dailymail\n\n!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!\nЯ ВЗЯВ 10 тис. даних з датасету через те, що у мене зависає середовище під час навчання та навіть\nперезавантажується комп'ютер + ЗМЕНШИВ КІЛЬКІСТЬ ТЕКСТІВ ДЛЯ СУМАРИЗАЦІЇ ДЛЯ ПАСИВНОГО ТА АКТИВНОГО НАВЧАННЯ.\n!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!\n\nЗапуск веб-додатку ідентичний з попередніх лаб.\n"

In [3]:
# Імпортуємо бібліотеки
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import torch
from transformers import BartForConditionalGeneration, BartTokenizer
from flask import Flask, request, render_template
import warnings
warnings.filterwarnings("ignore")

In [5]:
# Завантаження набору даних
df = pd.read_csv('train.csv').sample(n=10000, random_state=42)

print(df.head())

                                              id  \
272581  ed0fed726929c1eeabe6c390e47128dbb7d7a055   
772     023cd84001b33aed4ff0f3f5ecb0fdd2151cf543   
171868  6a70a0d8d3ed365fe1df6d35f1587a8b9b298618   
63167   b37204c13ea38b511265e41ac69fb12acfb63f85   
68522   c24e5805afd5145bc48410e876db91d44a06be5e   

                                                  article  \
272581  By . Mia De Graaf . Britons flocked to beaches...   
772     A couple who weighed a combined 32st were sham...   
171868  Video footage shows the heart stopping moment ...   
63167   Istanbul, Turkey (CNN) -- About 250 people rac...   
68522   By . Daily Mail Reporter . PUBLISHED: . 12:53 ...   

                                               highlights  
272581  People enjoyed temperatures of 17C at Brighton...  
772     Couple started piling on pounds after the birt...  
171868  A 17-year-old boy suffering lacerations to his...  
63167   Syrians citizens hightail it to Turkey .\nMost...  
68522   The Xue Long

In [7]:
# Підготовка даних
X = df['article'].values
y = df['highlights'].values

# Розділення даних на початковий тренувальний та тестовий набір
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Вибірка для активного навчання
X_pool, y_pool = shuffle(X_test, y_test, random_state=42)
initial_training_size = 50
active_iterations = 3
selection_size = 10

# Початковий тренувальний набір
X_initial, y_initial = X_pool[:initial_training_size], y_pool[:initial_training_size]
X_pool, y_pool = X_pool[initial_training_size:], y_pool[initial_training_size:]

In [9]:
# Завантаження моделі сумаризації з використанням Transformers
model_name = 'facebook/bart-large-cnn'
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

# Функція сумаризації тексту
def summarize(texts):
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=1024)
    summaries = model.generate(inputs['input_ids'], max_length=150, min_length=30, length_penalty=2.0, num_beams=4, early_stopping=True)
    summarized_texts = [tokenizer.decode(s, skip_special_tokens=True) for s in summaries]
    return summarized_texts

In [11]:
# Початкове навчання моделі та оцінка якості сумаризації
print("Початкове навчання моделі...")
initial_summaries = summarize(X_initial[:20].tolist())
print("Початкове навчання завершено. Оцінка на початковому наборі даних:")
for text, summary in zip(X_initial[:5], initial_summaries[:5]):
    print(f"Оригінал: {text[:100]}...")
    print(f"Підсумок: {summary}")
    print()

Початкове навчання моделі...
Початкове навчання завершено. Оцінка на початковому наборі даних:
Оригінал: By . Daily Mail Reporter . PUBLISHED: . 01:03 EST, 15 May 2013 . | . UPDATED: . 01:03 EST, 15 May 20...
Підсумок: Hussain Al Khawahir, 33, was detained after arriving at Detroit Metropolitan Airport on a flight from Amsterdam on Saturday. His lawyer said many people from Saudi Arabia travel with pressure cookers to cook lamb. Pressure cookers were set off at the finish line of the Boston Marathon on April 15.

Оригінал: ATLANTA, Georgia (CNN) -- The scar has faded over his left eye, but the fireworks injury that Tony W...
Підсумок: Over 9,000 people are injured each year because of fireworks, on average seven people die. Burns, lacerations and eye injuries are some of the most common injuries reported. Consumer Product Safety Commission: Never allow children to play with or ignite fireworks.

Оригінал: An American veterinarian's trip of a lifetime was ruined in September when she wa

In [13]:
"""
- Модель здатна зводити великі новинні тексти до основних фактів і ключових подій. 
Наприклад, у випадку з Арсеналом та FA Youth Cup, модель чітко виділила результат матчу та основні моменти, 
такі як переможний пенальті Деніела Кроулі. У підсумках добре передано основні факти.

- Модель добре справляється із завданням виділення суттєвих деталей у текстах на медичні або технічні теми, наприклад, 
як у випадку з інформацією про феєрверки.

- У деяких випадках модель могла не повністю зберегти контекст, що може призводити до втрати деталей або їх викривлення. 
Наприклад, у підсумку про жінок у США, інформація щодо регіонів була представлена нечітко

- Модель часом мала труднощі з правильним визначенням та передачею імен або географічних назв, 
що можна побачити в деяких підсумках. Наприклад, вона виявила деталі про Хуссейна Аль Хавагіра, 
але контекст того, як і чому його затримали, міг бути переданий точніше.

"""

'\n- Модель здатна зводити великі новинні тексти до основних фактів і ключових подій. \nНаприклад, у випадку з Арсеналом та FA Youth Cup, модель чітко виділила результат матчу та основні моменти, \nтакі як переможний пенальті Деніела Кроулі. У підсумках добре передано основні факти.\n\n- Модель добре справляється із завданням виділення суттєвих деталей у текстах на медичні або технічні теми, наприклад, \nяк у випадку з інформацією про феєрверки.\n\n- У деяких випадках модель могла не повністю зберегти контекст, що може призводити до втрати деталей або їх викривлення. \nНаприклад, у підсумку про жінок у США, інформація щодо регіонів була представлена нечітко\n\n- Модель часом мала труднощі з правильним визначенням та передачею імен або географічних назв, \nщо можна побачити в деяких підсумках. Наприклад, вона виявила деталі про Хуссейна Аль Хавагіра, \nале контекст того, як і чому його затримали, міг бути переданий точніше.\n\n'

In [15]:
# Імплементація активного навчання
for i in range(active_iterations):
    # Вибір найбільш невизначених прикладів для додавання до тренувального набору
    uncertain_indices = random.sample(range(len(X_pool)), selection_size)  # Використання випадкового вибору для спрощення
    
    # Додавання вибраних прикладів до тренувального набору
    X_selected = [X_pool[i] for i in uncertain_indices]
    y_selected = [y_pool[i] for i in uncertain_indices]
    
    X_initial = np.concatenate((X_initial, X_selected))
    y_initial = np.concatenate((y_initial, y_selected))
    
    # Видалення вибраних прикладів із пулу
    X_pool = np.delete(X_pool, uncertain_indices, axis=0)
    y_pool = np.delete(y_pool, uncertain_indices, axis=0)
    
    # Перенавчання моделі сумаризації (симуляція за допомогою нового сумаризування)
    summaries = summarize(X_initial[:20].tolist())  # Зменшено кількість текстів для сумаризації під час активного навчання
    print(f"Ітерація {i + 1}, Кількість текстів для навчання: {len(X_initial)}")

# Фінальна оцінка моделі
final_summaries = summarize(X_test[:5].tolist())  # Сумаризація для оцінки якості на тестовому наборі
print("Фінальна оцінка моделі на тестовому наборі:")
for text, summary in zip(X_test[:5], final_summaries):
    print(f"Оригінал: {text[:100]}...")
    print(f"Підсумок: {summary}")
    print()

Ітерація 1, Кількість текстів для навчання: 60
Ітерація 2, Кількість текстів для навчання: 70
Ітерація 3, Кількість текстів для навчання: 80
Фінальна оцінка моделі на тестовому наборі:
Оригінал: When the rest of us are steering clear of danger, Mike Hollingshead is seeking it out to document so...
Підсумок: Storm chaser Mike Hollingshead, from Nebraska, goes in search of tornadoes, supercells, lightning bolts and other natural phenomena throughout the United States' mid-west. The 38-year-old drives about 20,000 miles every year trying to find and shoot weather patterns that can rip up crops and blow houses away.

Оригінал: Martin Kaymer has vowed to follow in Rory McIlroy’s footsteps and use a bitter experience as added m...
Підсумок: Martin Kaymer threw away a 10-shot lead at the Abu Dhabi Championship. The German admits he has been doing plenty of soul-searching. Rory McIlroy experienced something similar at the Masters in 2011.

Оригінал: By . David Kent . Tim Sherwood launched an e

In [17]:
"""
- Активне навчання дало змогу моделі краще адаптуватися до складних текстів, на яких вона тренувалася в кожній ітерації, 
що видно з точніших підсумків у фінальній оцінці.

- Модель стала краще виявляти і виділяти ключові факти з оригінальних текстів. Наприклад, у випадку зі штормовим мисливцем Майком Голлінгсгедом, 
модель змогла виділити його діяльність, місце та типи погодних явищ, які він знімає, створюючи детальний і чіткий підсумок.

- Під кінець активного навчання модель змогла краще структуровано подавати підсумки текстів, зберігаючи їхню логіку і послідовність. 
Наприклад, підсумок про Тоттенхем і Тіма Шервуда був представлений послідовно і включав не лише результат, але й важливі коментарі тренера.

- Незважаючи на покращення, модель все ще має певні труднощі з повною передачею складних юридичних або кримінальних текстів. 
Наприклад, підсумок про п’ятьох колишніх охоронців Blackwater, хоча й містить важливу інформацію, іноді може виявляти тенденцію 
до втрати частини контексту.

"""

'\n- Активне навчання дало змогу моделі краще адаптуватися до складних текстів, на яких вона тренувалася в кожній ітерації, \nщо видно з точніших підсумків у фінальній оцінці.\n\n- Модель стала краще виявляти і виділяти ключові факти з оригінальних текстів. Наприклад, у випадку зі штормовим мисливцем Майком Голлінгсгедом, \nмодель змогла виділити його діяльність, місце та типи погодних явищ, які він знімає, створюючи детальний і чіткий підсумок.\n\n- Під кінець активного навчання модель змогла краще структуровано подавати підсумки текстів, зберігаючи їхню логіку і послідовність. \nНаприклад, підсумок про Тоттенхем і Тіма Шервуда був представлений послідовно і включав не лише результат, але й важливі коментарі тренера.\n\n- Незважаючи на покращення, модель все ще має певні труднощі з повною передачею складних юридичних або кримінальних текстів. \nНаприклад, підсумок про п’ятьох колишніх охоронців Blackwater, хоча й містить важливу інформацію, іноді може виявляти тенденцію \nдо втрати ча

In [19]:
# Створення веб-інтерфейсу для анотації за допомогою Flask
app = Flask(__name__)

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/annotate', methods=['POST'])
def annotate():
    # Отримання випадкового тексту з пулу для анотації
    if len(X_pool) > 0:
        index = random.choice(range(len(X_pool)))
        text = X_pool[index]
        return render_template('annotate.html', text=text, index=index)
    else:
        return "Пул текстів для анотації порожній."

@app.route('/submit_annotation', methods=['POST'])
def submit_annotation():
    index = int(request.form['index'])
    summary = request.form['summary']
    global X_initial, y_initial, X_pool, y_pool

    # Додавання анотації до тренувального набору
    X_initial = np.concatenate((X_initial, [X_pool[index]]))
    y_initial = np.concatenate((y_initial, [summary]))

    # Видалення тексту з пулу
    X_pool = np.delete(X_pool, index, axis=0)
    y_pool = np.delete(y_pool, index, axis=0)

    return "Анотація збережена, модель оновлена."

# Інтерфейс для перегляду і оцінки якості анотацій
@app.route('/review', methods=['GET'])
def review():
    # Перегляд випадкових 5 анотованих текстів
    random_indices = random.sample(range(len(X_initial)), min(5, len(X_initial)))
    annotated_texts = [(X_initial[i], y_initial[i]) for i in random_indices]
    return render_template('review.html', annotated_texts=annotated_texts)

if __name__ == '__main__':
    app.run(debug=True, port=5001, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
127.0.0.1 - - [25/Oct/2024 19:22:57] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2024 19:22:57] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [25/Oct/2024 19:23:06] "POST /annotate HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2024 19:23:33] "POST /submit_annotation HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2024 19:28:01] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2024 19:28:03] "GET /review HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2024 19:28:13] "GET /review HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2024 19:29:02] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2024 19:29:05] "GET /review HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2024 19:34:00] "POST /annotate HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2024 19:35:06] "GET /review HTTP/1.1" 200 -
